# Тестирование функций и методов
---

### Модуль Ingestion (Docling)

#### Загрузка и конвертация документа

In [ ]:
import src.document_loader as document_loader
import config.config as cfg

def test_load_document():
    file_path = str(cfg.RAW_DATA_DIR / "sample.pdf")
    result = document_loader.load_document(file_path)
    print(f"Result_doc: {result[0:100]}...")  # Print first 100 characters for brevity

if __name__ == "__main__":
    test_load_document()

[INFO] 2026-01-23 14:02:37,881 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2026-01-23 14:02:37,882 [RapidOCR] device_config.py:50: Using CPU device
[INFO] 2026-01-23 14:02:37,893 [RapidOCR] download_file.py:60: File exists and is valid: D:\Projects\LocalRAG\.venv\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.pth
[INFO] 2026-01-23 14:02:37,895 [RapidOCR] main.py:50: Using D:\Projects\LocalRAG\.venv\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.pth
[INFO] 2026-01-23 14:02:38,205 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2026-01-23 14:02:38,205 [RapidOCR] device_config.py:50: Using CPU device
[INFO] 2026-01-23 14:02:38,206 [RapidOCR] download_file.py:60: File exists and is valid: D:\Projects\LocalRAG\.venv\Lib\site-packages\rapidocr\models\ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2026-01-23 14:02:38,206 [RapidOCR] main.py:50: Using D:\Projects\LocalRAG\.venv\Lib\site-packages\rapidocr\models\ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2026-01-23 

Saved intermediate file: data\processed\sample.md
Result_doc: 1. Научные открытия и технологические прорывы

В 2023 году учёные из ЦЕРНа подтвердили существование...


#### Чанкирование по кол-ву симовлов с перекрытием

In [1]:
from src.chunker import chunk_text
from config.config import PROCESSED_DATA_DIR

file_path = PROCESSED_DATA_DIR / "sample.md"
markdown_content = file_path.read_text(encoding="utf-8")

CHUNK_SIZE = 500 
OVERLAP = 100
file_name = file_path.name # Сохраняем имя для Qdrant

chunks = chunk_text(markdown_content, chunk_size=CHUNK_SIZE, overlap=OVERLAP)

print(f"Файл '{file_name}' успешно разбит.")
print(f"Количество чанков: {len(chunks)}")
print(f"Пример первого чанка:\n{repr(chunks[0][:100])}...")

Файл 'sample.md' успешно разбит.
Количество чанков: 12
Пример первого чанка:
'1. Научные открытия и технологические прорывы\n\nВ 2023 году учёные из ЦЕРНа подтвердили существование'...


#### Сохранение векторизованных данных в Qdrant

In [2]:
from src.embedder import vectorize_and_upload

q_client, embed_model = vectorize_and_upload(chunks, file_name)

d:\Projects\LocalRAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Генерация векторов для 12 чанков...
Данные успешно загружены в коллекцию 'docs_collection'


### Модули Retrieval и Reranker

#### Retrieval

In [3]:
from src.retrieval import retrieve

user_query = "Скольки градусам может достичь потепление к 2050 году?"
candidates = retrieve(user_query, q_client, embed_model, top_k=3)

Найдено 3 кандидатов через векторный поиск.


In [4]:
print("Топ-3 кандидата по запросу:")
for i, candidate in enumerate(candidates, start=1):
    print(f"{i}. {candidate}\n")

Топ-3 кандидата по запросу:
1. {'text': 'аду МГЭИК (Межправительственной группы экспертов по изменению климата) за 2022 год, глобальная средняя температура уже на 1,15°C выше доиндустриального уровня. Если выбросы парниковых газов не будут радикально сокращены, к 2050 году потепление может достичь 2,7°C, что приведёт к катастрофическим последствиям: повышению уровня моря, экстремальным погодным явлениям и массовому вымиранию видов.\n\nОднако существуют и контраргументы: некоторые исследователи, такие как климатолог Бьорн Ломборг, утвержд', 'score': 0.7725014352780308, 'metadata': {'source': 'sample.md', 'chunk_id': 5}}

2. {'text': 'ко существуют и контраргументы: некоторые исследователи, такие как климатолог Бьорн Ломборг, утверждают, что адаптация к изменениям климата может быть более эффективной стратегией, чем полный отказ от ископаемого топлива. Тем не менее, большинство научного сообщества поддерживает Парижское соглашение 2015 года, целью которого является удержание роста темпер

#### Reranker

In [5]:
from src.reranker import rerank

final_context = rerank(user_query, candidates, top_n=3)

# Вывод результата
for i, res in enumerate(final_context):
    print(f"\n--- Релевантный чанк №{i+1} (Score: {res['rerank_score']:.4f}) ---")
    print(res["text"][:200] + "...")

Переранжирование завершено. Выбрано топ-3 наиболее релевантных.

--- Релевантный чанк №1 (Score: 8.6397) ---
аду МГЭИК (Межправительственной группы экспертов по изменению климата) за 2022 год, глобальная средняя температура уже на 1,15°C выше доиндустриального уровня. Если выбросы парниковых газов не будут р...

--- Релевантный чанк №2 (Score: 8.4987) ---
 содержащих стереотипы, она может их воспроизводить.

В то же время ИИ активно применяется в медицине: алгоритмы DeepMind (подразделение Google) помогают диагностировать диабетическую ретинопатию и ра...

--- Релевантный чанк №3 (Score: 7.9926) ---
ко существуют и контраргументы: некоторые исследователи, такие как климатолог Бьорн Ломборг, утверждают, что адаптация к изменениям климата может быть более эффективной стратегией, чем полный отказ от...


In [6]:
final_context[0]["text"]

'аду МГЭИК (Межправительственной группы экспертов по изменению климата) за 2022 год, глобальная средняя температура уже на 1,15°C выше доиндустриального уровня. Если выбросы парниковых газов не будут радикально сокращены, к 2050 году потепление может достичь 2,7°C, что приведёт к катастрофическим последствиям: повышению уровня моря, экстремальным погодным явлениям и массовому вымиранию видов.\n\nОднако существуют и контраргументы: некоторые исследователи, такие как климатолог Бьорн Ломборг, утвержд'

### Модуль Generator

#### Генератор с использованием локальной модели

In [8]:
from src.generator import Generator

generator = Generator()

user_query = "Скольки градусам может достичь потепление к 2050 году?"
response = generator.generate(user_query, final_context[0]["text"])

print("\n--- Сгенерированный ответ ---")
print(response)


--- Сгенерированный ответ ---
К 2050 году потепление может достичь **2,7°C**, если выбросы парниковых газов не будут радикально сокращены.


Получение итогового списка контекста модели

In [9]:
final_context_text = []
for i in range(len(final_context)):
    final_context_text.append(final_context[i]["text"])

In [10]:
print(final_context_text)

['аду МГЭИК (Межправительственной группы экспертов по изменению климата) за 2022 год, глобальная средняя температура уже на 1,15°C выше доиндустриального уровня. Если выбросы парниковых газов не будут радикально сокращены, к 2050 году потепление может достичь 2,7°C, что приведёт к катастрофическим последствиям: повышению уровня моря, экстремальным погодным явлениям и массовому вымиранию видов.\n\nОднако существуют и контраргументы: некоторые исследователи, такие как климатолог Бьорн Ломборг, утвержд', ' содержащих стереотипы, она может их воспроизводить.\n\nВ то же время ИИ активно применяется в медицине: алгоритмы DeepMind (подразделение Google) помогают диагностировать диабетическую ретинопатию и рак молочной железы с точностью, сопоставимой с экспертами-людьми.\n\n## 5. География и демография\n\nНаселение Земли достигло 8 миллиардов человек в ноябре 2022 года. По прогнозам ООН, пик будет достигнут около 2086 года - на уровне 10,4 миллиарда. При этом старение населения становится сер

---
### Evaluation (Ragas)

#### Полный цикл оценки (Full Pipeline Evaluation)
Включает в себя подготовку данных, генерацию ответов и расчет метрик: Faithfulness, Answer Relevancy, Context Precision, Context Recall.

In [11]:
from src.evaluation.ragas_eval import RagasEvaluator

# Инициализация Ragas Evaluator
evaluator = RagasEvaluator()
print("Ragas Evaluator initialized successfully.")

d:\Projects\LocalRAG\src\evaluation\ragas_eval.py:32: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceEmbeddings(model_name=cfg.EMBEDDING_MODEL)


Ragas Evaluator initialized successfully.


In [12]:
# 1. Подготовка тестовых данных (Questions & Ground Truths)
test_questions = [
    "Скольки градусам может достичь потепление к 2050 году?",
    "Какие алгоритмы DeepMind используются в медицине?",
    "Каков прогноз численности населения Земли на 2086 год?"
]

ground_truths = [
    "Потепление может достичь 2,7°C.",
    "Алгоритмы DeepMind помогают диагностировать диабетическую ретинопатию и рак молочной железы.",
    "По прогнозам ООН, пик будет достигнут около 2086 года на уровне 10,4 миллиарда человек."
]

# 2. Запуск пайплайна (Inference)
answers = []
contexts = []

print("Запуск генерации ответов...")
for q in test_questions:
    # Retrieve
    candidates = retrieve(q, q_client, embed_model, top_k=5)
    final_c = rerank(q, candidates, top_n=3)
    
    # Extract context texts
    c_texts = [c["text"] for c in final_c]
    combined_c = "\n\n".join(c_texts)
    
    # Generate
    resp = generator.generate(q, combined_c)
    
    answers.append(resp)
    contexts.append(c_texts)
    print(f"Q: {q}\nA: {resp[:100]}...\n")



Запуск генерации ответов...
Найдено 5 кандидатов через векторный поиск.
Переранжирование завершено. Выбрано топ-3 наиболее релевантных.
Q: Скольки градусам может достичь потепление к 2050 году?
A: К 2050 году, если выбросы парниковых газов не будут радикально сокращены, потепление может достичь *...

Найдено 5 кандидатов через векторный поиск.
Переранжирование завершено. Выбрано топ-3 наиболее релевантных.
Q: Какие алгоритмы DeepMind используются в медицине?
A: Алгоритмы DeepMind, упомянутые в контексте, применяются в медицине для диагностики диабетической рет...

Найдено 5 кандидатов через векторный поиск.
Переранжирование завершено. Выбрано топ-3 наиболее релевантных.
Q: Каков прогноз численности населения Земли на 2086 год?
A: Прогноз численности населения Земли на 2086 год, согласно контексту, составляет **10,4 миллиарда чел...



In [13]:
# 4. Запуск оценки
print("Запуск оценки Ragas через модуль src.evaluation...")
df_results = evaluator.run_evaluation(
    questions=test_questions,
    answers=answers,
    contexts=contexts,
    ground_truths=ground_truths
)

print("\nРезультаты оценки:")
display(df_results)

# Сохранение результатов
evaluator.save_results(df_results, "data/evaluation_results/evaluation_results.csv")

Запуск оценки Ragas через модуль src.evaluation...
Starting Ragas evaluation pipeline...


Evaluating: 100%|██████████| 12/12 [05:17<00:00, 26.47s/it]



Результаты оценки:


,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_precision,context_recall
0,Скольки градусам может достичь потепление к 20...,[аду МГЭИК (Межправительственной группы экспер...,"К 2050 году, если выбросы парниковых газов не ...","Потепление может достичь 2,7°C.",1.000000,0.862986,1.000000,1.0
1,Какие алгоритмы DeepMind используются в медицине?,"[ содержащих стереотипы, она может их воспроиз...","Алгоритмы DeepMind, упомянутые в контексте, пр...",Алгоритмы DeepMind помогают диагностировать ди...,1.000000,0.702630,1.000000,1.0
2,Каков прогноз численности населения Земли на 2...,[аду МГЭИК (Межправительственной группы экспер...,Прогноз численности населения Земли на 2086 го...,"По прогнозам ООН, пик будет достигнут около 20...",0.666667,0.999443,0.333333,1.0


Results saved to data/evaluation_results/evaluation_results.csv


## Сравнительное тестирование методов чанкирования
---
В этом разделе мы запускаем полный цикл RAG (Ingestion -> Chunking -> Embedding -> Retrieval -> Reranking -> Generation -> Evaluation) для каждого из 5 реализованных методов чанкирования, чтобы сравнить их эффективность.


In [ ]:
# Импорты необходимых модулей
import pandas as pd
from src.chunkers.recursive_chunker import chunk_recursive
from src.chunkers.token_chunker import chunk_token
from src.chunkers.markdown_chunker import chunk_markdown
from src.chunkers.sentence_window_chunker import chunk_sentence_window
from src.chunkers.semantic_chunker import chunk_semantic

from src.embedder import vectorize_and_upload
from src.retrieval import retrieve
from src.reranker import rerank
from src.generator import Generator
from src.evaluation.ragas_eval import RagasEvaluator
from config.config import PROCESSED_DATA_DIR

# Загрузка текста
file_path = PROCESSED_DATA_DIR / "sample.md"
markdown_content = file_path.read_text(encoding="utf-8")
print(f"Текст загружен, длина: {len(markdown_content)} символов")

# Инициализация общих компонентов
generator = Generator()
evaluator = RagasEvaluator()
print("Компоненты инициализированы.")


In [ ]:
# Тестовые данные (Вопросы и эталонные ответы)
test_questions = [
    "Скольки градусам может достичь потепление к 2050 году?",
    "Какие алгоритмы DeepMind используются в медицине?",
    "Каков прогноз численности населения Земли на 2086 год?"
]

ground_truths = [
    "Потепление может достичь 2,7°C.",
    "Алгоритмы DeepMind помогают диагностировать диабетическую ретинопатию и рак молочной железы.",
    "По прогнозам ООН, пик будет достигнут около 2086 года на уровне 10,4 миллиарда человек."
]


In [ ]:
def run_pipeline_for_method(method_name, chunk_func, text, questions, ground_truths):
    print(f"\n{'='*20} Testing Method: {method_name} {'='*20}")
    
    # 1. Chunking
    print("1. Chunking...")
    try:
        # Некоторые методы могут требовать доп. параметров, но мы используем дефолтные для теста
        chunks = chunk_func(text)
        print(f"   Получено {len(chunks)} чанков.")
    except Exception as e:
        print(f"   Ошибка при чанкировании: {e}")
        return None
        
    if not chunks:
        print("   Чанки не созданы.")
        return None

    # 2. Embedding & Upload (Unique Collection)
    collection_name = f"docs_{method_name.lower().replace(' ', '_')}"
    print(f"2. Embedding to collection '{collection_name}'...")
    client, embed_model = vectorize_and_upload(chunks, "sample.md", collection_name=collection_name)

    # 3. Retrieval, Reranking, Generation
    print("3. Running RAG Pipeline (Retrieve -> Rerank -> Generate)...")
    answers = []
    contexts = []
    
    for q in questions:
        # Retrieve
        candidates = retrieve(q, client, embed_model, top_k=5, collection_name=collection_name)
        # Rerank
        final_c = rerank(q, candidates, top_n=3)
        
        # Extract context texts
        c_texts = [c["text"] for c in final_c]
        combined_c = "\n\n".join(c_texts)
        
        # Generate
        resp = generator.generate(q, combined_c)
        answers.append(resp)
        contexts.append(c_texts)
        # print(f"   Q: {q[:30]}... -> A: {resp[:30]}...")

    # 4. Evaluation
    print("4. Evaluating...")
    df_results = evaluator.run_evaluation(
        questions=questions,
        answers=answers,
        contexts=contexts,
        ground_truths=ground_truths
    )
    
    # Add method column
    df_results['method'] = method_name
    return df_results


In [ ]:
# Список методов для тестирования
methods = [
    ("Recursive", chunk_recursive),
    ("Token", chunk_token),
    ("Markdown", chunk_markdown),
    ("Sentence Window", chunk_sentence_window),
    ("Semantic", chunk_semantic)
]

all_results = []

for name, func in methods:
    df = run_pipeline_for_method(name, func, markdown_content, test_questions, ground_truths)
    if df is not None:
        all_results.append(df)

# Объединение результатов
if all_results:
    final_df = pd.concat(all_results, ignore_index=True)
    print("\nВсе тесты завершены.")
else:
    print("\nНет результатов.")


### Анализ результатов
Ниже приведена сводная таблица средних метрик по каждому методу чанкирования.


In [ ]:
if all_results:
    # Группировка по методу и вычисление среднего
    numeric_cols = ['faithfulness', 'answer_relevancy', 'context_precision', 'context_recall']
    summary = final_df.groupby('method')[numeric_cols].mean()
    
    print("Сводная таблица (средние значения):")
    display(summary)
    
    # Сохранение полных результатов
    final_df.to_csv("data/evaluation_results/chunking_comparison.csv", index=False)
    print("\nПолные результаты сохранены в data/evaluation_results/chunking_comparison.csv")
